In [1]:
import sys
fileDir = "/home/jovyan/notebooks/"
sys.path.append(fileDir)

from utilities import *

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
session = create_spark_session("Neo4j GitHub", SparkConnector.NEO4J)

Added dependencies: 
 ['neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar']


In [ ]:
# Scenario 1

options = get_default_options(SparkConnector.NEO4J)
options["query"] =  """
                        MATCH (contrib:GitContributor)-[:AUTHOR]->(commit:GitCommit)-[:BELONGS_TO]->(repo:GitRepository)
                        WITH contrib, COUNT(DISTINCT repo) as repo_count
                        RETURN contrib, repo_count ORDER BY repo_count DESC
                    """
top10contributors = spark_read(SparkConnector.NEO4J, session, options=options)
display(top10contributors.take(10))

In [ ]:
# Scenario 2:

LANGUAGE = "C++"
PERCENTAGE = 0.5
options = get_default_options(SparkConnector.NEO4J)
options["query"] = f"""
                    MATCH (r:GitRepository)-[w:WRITTED_IN]->(l:GitLanguage)
                    WITH r, SUM(w.bytes) AS totalBytesForRepo, collect({{language_name:l.name,bytes: w.bytes}}) AS bytesForLanguages
                    UNWIND bytesForLanguages AS bytesForLanguage
                    WITH r.name AS repo_name, bytesForLanguage.language_name AS lang, round((bytesForLanguage.bytes*1.0/totalBytesForRepo),2) AS percOfBytes
                        WHERE lang = "{LANGUAGE}" AND percOfBytes > {PERCENTAGE}
                    RETURN repo_name, lang, percOfBytes 
                  """
bytesPercentageInRepos = spark_read(SparkConnector.NEO4J, session, options=options)
display(bytesPercentageInRepos.take(10))

In [ ]:
# Scenario 3:
REPO_NAME = "tensorflow/tensorflow"
options = get_default_options(SparkConnector.NEO4J)
options["query"] =  f"""
                        MATCH (repository:GitRepository {{name: "{REPO_NAME}"}})<-[:BELONGS_TO]-(commit:GitCommit), 
                            r = (commit)-[:PARENT]->()
                        WITH commit, collect(r) AS parents
                        WHERE size(parents) > 1
                        RETURN count(commit) AS mergeCount
                    """
bytesPercentageInRepos = spark_read(SparkConnector.NEO4J, session, options=options)
bytesPercentageInRepos.show(10)

In [ ]:
# Scenario 4 - project graph contributors, commits and repos:
query = """     
        CALL gds.graph.project.cypher(
              'contribRepoAndCommits',
              'MATCH (n) WHERE n:GitContributor OR n:GitCommit OR n:GitRepository RETURN ID(n) AS id',
              'MATCH (n)-[r]->(m) WHERE r:PARENT OR r:BELONGS_TO OR r:COMMITTED OR r:AUTHOR RETURN ID(n) AS source, ID(m) AS target'
        )
        """
raise Exception("Put query text in the neo4j GUI")

In [8]:
# Scenario 4 - LabelPropagation:
PROJ_NAME = "contribRepoAndCommits"  # "repoAndCommits"

options = get_default_options(SparkConnector.NEO4J)
options["query"] =  f"""
                        CALL gds.labelPropagation.stream('{PROJ_NAME}')
                        YIELD nodeId, communityId
                        RETURN nodeId as ID, communityId 
                    """
louvain = spark_read(SparkConnector.NEO4J, session, options=options)
louvain.write.option("header", True).mode("overwrite").csv("hdfs://namenode:9000//data-team/labelPropagation")

Dataframe loaded from neo4j


In [ ]:
session.sparkContext.stop()
session.stop()